# chapter 5 - Recommendation Systems
추천 시스템
1. 협업 필터링 (CF) Collaborative Filtering 
2. 콘텐츠 기반 필터링 (CBF) Content-based Filtering

기타 접근방법
1. 연관 룰 association rules
2. $Log$ 우도 the log-likelihood method
3. 하이브리드 hybrid methods

<br></br>
## 1 유틸리티 행렬
Utility matrix 추천시스템에 사용되는 계량척도 시스템

평점 사용자 $i$가 아이템 $j$를 평가한 사용자 평가목록 : $ r_{ij}$ 

<img src="https://image.slidesharecdn.com/4-recommendationcolaborativefiltering-150818025112-lva1-app6892/95/recommender-systems-contentbased-and-collaborative-filtering-9-638.jpg?cb=1439866428" align='left' width='300'>

### 01 데이터 수집 및 전처리
import Csv files 

In [1]:
import numpy as np
import pandas as pd
from time import time
from scipy import linalg
from collections import defaultdict
import copy, collections, math

# 사용자 평점 데이터 불러오기
df = pd.read_csv('./data/ml-100k/u.data', sep='\t', header=None)
movies_rated  = list(df[1]) 
counts = collections.Counter(movies_rated)
df.head(2)

,0,1,2,3
0,196,242,3,881250949
1,186,302,3,891717742


In [2]:
# https://stackoverflow.com/questions/18171739/unicodedecodeerror-when-reading-csv-file-in-pandas-with-python
# 데이터 영화목록 불러오기
df_info = pd.read_csv('./data/ml-100k/u.item', 
                      sep='|', header=None, encoding = "ISO-8859-1")
# 영화목록 추출 (index 라인 번호를 제목 뒤에 덧붙임)
movie_list = [df_info[1].tolist()[ index_num ] + ';' + str(index_num+1) 
              for index_num in range(len(df_info[1].tolist())) ]

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [3]:
df_out = pd.DataFrame(columns=['user']+movie_list)
print(df_out.shape)
df_out.ix[:,:6]  # 영화의 목록을 컬럼으로 생성

(0, 1683)


,user,Toy Story (1995);1,GoldenEye (1995);2,Four Rooms (1995);3,Get Shorty (1995);4,Copycat (1995);5


In [4]:
min_ratings = 50  # 작성자 리뷰중 50개 이하만 기록된 영화 데이터는 제거
num_users  = len(df[0].drop_duplicates().tolist()) # 전체 사용자 수  
num_movies = len(movie_list) # 전체 영화 수


In [5]:
to_removelist = []; t0 = time()
for i in range(1, num_users):
    tmp_movielist = [0 for j in range(num_movies)]
    df_tmp = df[df[0] == i]

    for k in df_tmp.index:
        if counts[ df_tmp.ix[k][1] ] >= min_ratings:
            tmp_movielist[ df_tmp.ix[k][1] -1 ] = df_tmp.ix[k][2]
        else: to_removelist.append(df_tmp.ix[k][1])
    df_out.loc[i] = [i] + tmp_movielist   # df의 사용자 리뷰 데이터를 df_out 에 첨부

to_remove_list = list(set(to_removelist))
df_out.drop(df_out.columns[ to_removelist ], axis = 1, inplace = True)
df_out.to_csv('data/utilitymatrix.csv', index = None)
print(round(time()-t0,4),'sec'); print(df_out.shape); df_out.iloc[:3,:7]

32.9189 sec
(942, 604)


,user,Toy Story (1995);1,GoldenEye (1995);2,Four Rooms (1995);3,Get Shorty (1995);4,Copycat (1995);5,Twelve Monkeys (1995);7
1,1.0,5.0,3.0,4.0,3.0,3.0,4.0
2,2.0,4.0,0.0,0.0,0.0,0.0,0.0
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# 01 내용 정리
# 총 1683개의 영화에 대한 개별 사용자의 평점을 첨부한다
# 50개 이하의 평점이 기록된 영화는 제거되어, 최종 604개의 영화기록이 정제된다

### 02 결측치 처리
NaN 결측치는, '대체'(imputation)의 방법으로 초기화 한다

대체 (imputation) : <strong>사용자의 평균 평점</strong>과 <strong>영화별 평균 평점</strong>을 대체에 사용한다.

In [7]:
def imputation(inp,Ri):
    Ri = Ri.astype(float)
    def userav():
        for i in range(len(Ri)):
            Ri[i][Ri[i] == 0] = sum( Ri[i] ) / float(len(Ri[i][Ri[i] > 0]))
        return Ri
    def itemav():
        for i in range(len(Ri[0])):
            Ri[:, i][Ri[:, i] == 0] = sum(Ri[:,i])/float(len(Ri[:, i][Ri[:, i] > 0]))
        return Ri            
    switch = {'useraverage' : userav() ,'itemaverage' : itemav() }
    return switch[inp]

In [8]:
# 유틸리티 행렬 R은
# 사용자는 N명이고, 아이템은 M개가 된다

<br></br>
## 2 유사도 척도
벡터 $X$와 $Y$의 유사도를 계산하기 위한 척도
1. 코사인 유사도 Cosine similarity 
$$ s(x,y) = \frac{\sqrt{\sum x_i y_i}}{\sqrt{\sum x_i^2}\sqrt{\sum y_i^2}} $$
2. 피어슨 상관관계 Pearson correlation
$$ s(x,y) = \frac{\sqrt{\sum (x_i - \bar x)(y_i - \bar y)}}
{\sqrt{\sum (x_i - \bar x)^2}\sqrt{\sum (y_i - \bar y)^2}} $$

두 척도의 평균이 0인경우, 일치하게 된다

In [9]:
# scipy를 활용한 유사도 척도 함수
from scipy.stats import pearsonr
from scipy.spatial.distance import cosine 
def similarity(x,y,metric='cos'):
    if metric == 'cos':
        return 1.-cosine(x,y)   # 코사인 유사도를 출력
    else:
        return pearsonr(x,y)[0] # 피어슨 상관계수를 출력

<br></br>
## 3 협업 필터링 방법
<h4><strong>Collaborative Filtering methods</strong></h4>
"자신과 비슷한 사람들은, 좋아하는 아이템을 공유한다"를 기반으로 한다

이는 사용자의 수많은 데이터를 필요로 해서 발생하는 <strong>Cold Start</strong>

(반대로 <strong>가용 데이터 부족</strong>의 <strong>Warm Start</strong>가 있다)가 문제가 되고 있다

이를 극복하기 위하여, <strong>CF</strong>와 <strong>CBF의 하이브리드</strong> 방식이 제안된다

(콘텐츠의 내부적 분류로써, 사용자의 데이터를 더 세분화 한 데이터로 분석을 한다)
<h4><strong>콘텐츠 기반 알고리즘의 이슈</strong></h4>
공통이슈로 <strong>'확장성(Scalability)'</strong>이 언급되는데

이는 사용자와 아이템 갯수에 비례하여 연산량이 급증하는 문제를 말한다(<strong>병렬처리</strong>가 필요)

반면 아이템 갯수가 작은경우에는 <strong>희소성(Sparsity)</strong> 문제가 생긴다(<strong>대체</strong>의 방법으로 해결


<h3>01 <strong>메모리 기반의 협업 필터링</strong> Memory-based Collaborative Filtering</h3> 
유틸리티 행렬을 사용하여 <strong>아이템의 유사도</strong>를 계산한다

<h4>1) <strong>사용자 기반 협업 필터링</strong> User-based Collaborative Filtering</h4>
<strong> K-NN 알고리즘</strong>으로, 유사 사용자들의 평점을 찾은 뒤 

<strong>가중 평균</strong>으로,누락 평점 대신에 사용된다
1. 사용자 $i$와 <strong>평가되지 않은 아이템 $j$</strong>를 특정
2. <strong>유사도 척도 $s$</strong>를 사용, $j$아이템에 유사평점을 준 사용자 $K$를 찾는다 ($20 < K < 50$)
3. $K$들의 평점을 <strong>가중평균</strong>하여, 사용자 $i$의 평점을 예측한다
4. 균질적인 평점을 비교하기 위하여, <strong>평점분포를 정규화</strong> 한다
5. 예측평균이 1~5 기본 척도값 이외가 나온경우, 최소값은 1, 최대값은 5로 조정

In [10]:
def CF_userbased(u_vec, K, data, indxs = False):
    def FindKNeighbours(r, data, K):
        neighs, cnt = [], 0
        for u in range(len(data)):
            if data[u,r] > 0  and  cnt < K:
                neighs.append(data[u])
                cnt += 1 
            elif cnt == K: break
        return np.array(neighs)

    def CalcRating(u_vec, r, neighs):
        rating, den = 0., 0.
        for j in range(len(neighs)):
            rating += neighs[j][-1] * float(neighs[j][r] - neighs[j][neighs[j] > 0][:-1].mean())
            den += abs(neighs[j][-1])
        if den > 0:    rating = np.round(u_vec[u_vec>0].mean()+(rating/den),0)
        else:          rating = np.round(u_vec[u_vec>0].mean(),0)
        if rating > 5: return 5.
        elif rating<1: return 1.
        return rating 

    data = data.astype(float)
    nrows, ncols = len(data), len(data[0])
    data_sim = np.zeros((nrows,ncols+1))
    data_sim[:,:-1] = data

    # calc similarities:
    for u in range(nrows):
        if np.array_equal(data_sim[u, :-1], u_vec) == False: 
            data_sim[u,ncols] = sim(data_sim[u,:-1],u_vec,'pearson')
        else:
            data_sim[u,ncols] = 0.

    #order by similarity:
    data_sim =data_sim[data_sim[:,ncols].argsort()][::-1]
    #find the K users for each item not rated:
    u_rec = np.zeros(len(u_vec))
    for r in range(ncols):
        if u_vec[r]==0:
            neighs = FindKNeighbours(r,data_sim,K)
            # calc the predicted ratin
            u_rec[r] = CalcRating(u_vec,r,neighs)

    #take out the rated movies
    if indxs:
        seenindxs = [indx for indx in range(len(u_vec)) if u_vec[indx]>0]
        u_rec[seenindxs] = -1
        recsvec = np.argsort(u_rec)[::-1][np.argsort(u_rec)>0]
        return recsvec    
    return u_rec

In [11]:
# 위 함수 사용해본 결과
# 피어슨 상관관계가 조금 더 결과가 좋은데, 사용자의 평균평점을 제외한채 사용하기 때문으로 보인다

<h4>2) <strong>아이템 기반의 협업 필터링</strong> Item-based Collaborative Filtering</h4> 
<strong>아이템에 대해 유사도가 측정</strong>된다는 점을 제외하고는 개념적으로, 사용자기반과 동일하다
1. <strong>유사도 측정치 $s$</strong>를 사용하여, 사용자 $i$가 평가한 아이템과 가장 <strong>비슷한 아이템을 $K$개</strong> 찾는다
2. $K$개 아이템 평점을 가중평균하여 예측평점으로 계산한다
$$ P_{ij} = \frac{\sum S(j,k)r_{ik}}{|\sum{S(j,k)}|} $$

유사도 측정결과 음수도 가능하므로, <strong>가중 평균의 유효성을 높이기 위해서</strong> 양수값들만의 가중평균을 사용하면 더 효과적이다

In [12]:
class CF_itembased(object):
    def __init__(self,data):
        #calc item similarities matrix
        nitems = len(data[0])
        self.data = data
        self.simmatrix = np.zeros((nitems,nitems))
        for i in range(nitems):
            for j in range(nitems):
                if j>=i: # tri-angular matrix
                    self.simmatrix[i,j] = sim(data[:,i],data[:,j])
                else: self.simmatrix[i,j] = self.simmatrix[j,i]

    def GetKSimItemsperUser(self,r,K,u_vec):
        items = np.argsort(self.simmatrix[r])[::-1]
        items = items[items!=r]
        cnt=0
        neighitems = []
        for i in items:
            if u_vec[i]>0 and cnt<K:
                neighitems.append(i)
                cnt+=1
            elif cnt==K: break
        return neighitems
        
    def CalcRating(self,r,u_vec,neighitems):
        rating = 0.
        den = 0.
        for i in neighitems:
            rating +=  self.simmatrix[r,i]*u_vec[i]
            den += abs(self.simmatrix[r,i])
        if den>0:
            rating = np.round(rating/den,0)
        else:
            rating = np.round(self.data[:,r][self.data[:,r]>0].mean(),0)
        return rating
        
    def CalcRatings(self,u_vec,K,indxs=False):
        #u_rec = copy.copy(u_vec)
        u_rec = np.zeros(len(u_vec))
        for r in range(len(u_vec)):
            if u_vec[r]==0:
                neighitems = self.GetKSimItemsperUser(r,K,u_vec)
                #calc predicted rating
                u_rec[r] = self.CalcRating(r,u_vec,neighitems)
        if indxs:
            #take out the rated movies
            seenindxs = [indx for indx in range(len(u_vec)) if u_vec[indx]>0]
            u_rec[seenindxs]=-1
            recsvec = np.argsort(u_rec)[::-1][np.argsort(u_rec)>0]        
            return recsvec
        return u_rec

In [13]:
# 클래스 내용 정리
# simmatrix로 아이템 유사도 행렬을 계산한다
# 아이템의 사용자 평점이 없는 경우, CalcRatings로 '가중평균 평점'을 예측을 한다
# GetKSimItemsperUsers는 '사용자가 평가하지 않은 아이템과 유사도가 높은 아이템' 중
# 사용자가 과거에 평가했던 K개의 아이템을 찾는다
# 이웃을 찾을 수 없는 경우는, 아이템의 평균평점으로 설정한다
# u_vec 는 사용자 평점 벡터로, 유틸리티 행렬의 행 벡터다

<h4>3) <strong>단순 아이템 기반의 협업 필터링 : 슬롭원 </strong>Slope one </h4>
1. 아이템 $j$와 가장 차이가 작은 아이템 $K$를 찾는다
2. 예측 평점을 가중평균으로 계산한다
$$ P_{ij} = \frac{ \sum(d_{jk}+ r_{jk})\sum n_{jk}^l} {\sum \sum_{i=1}^N n_{jk}^l } $$

CF알고리즘에 비해 변수를 간략하게 조정하여 단순하지만, 대체로 정확하고 계산비용이 낮아 구현에 용이하다

In [14]:
class SlopeOne(object):
    def __init__(self,Umatrix):
        #calc item similarities matrix
        nitems = len(Umatrix[0])
        self.difmatrix = np.zeros((nitems,nitems))
        self.nratings = np.zeros((nitems,nitems))
        def diffav_n(x,y):
            xy = np.vstack((x, y)).T
            xy = xy[(xy[:,0]>0) & (xy[:,1]>0)]
            nxy = len(xy)
            if nxy == 0:
                #print 'no common'
                return [1000.,0]
            return [float(sum(xy[:,0])-sum(xy[:,1]))/nxy,nxy]
            
        for i in range(nitems):
            for j in range(nitems):
                if j>=i: #triangular matrix                 
                    self.difmatrix[i,j],self.nratings[i,j] = diffav_n(Umatrix[:,i],Umatrix[:,j])
                else:
                    self.difmatrix[i,j] = -self.difmatrix[j,i]
                    self.nratings[i,j] = self.nratings[j,i]
        
    def GetKSimItemsperUser(self,r,K,u_vec):
        items = np.argsort(self.difmatrix[r])
        items = items[items!=r]
        cnt=0
        neighitems = []
        for i in items:
            if u_vec[i]>0 and cnt<K:
                neighitems.append(i)
                cnt+=1
            elif cnt==K: break
        return neighitems
        
    def CalcRating(self,r,u_vec,neighitems):
        rating = 0.
        den = 0.
        for i in neighitems:
            if abs(self.difmatrix[r,i])!=1000:
                rating +=  (self.difmatrix[r,i]+u_vec[i])*self.nratings[r,i]
                den += self.nratings[r,i]       
        #print 'no similar diff'
        if den==0: return 0.            
        rating = np.round(rating/den,0)
        if rating >5:    return 5.
        elif rating <1.: return 1.
        return rating
        
    def CalcRatings(self,u_vec,K):
        #u_rec = copy.copy(u_vec)
        u_rec = np.zeros(len(u_vec))
        for r in range(len(u_vec)):
            if u_vec[r]==0:
                neighitems = self.GetKSimItemsperUser(r,K,u_vec)
                # calc predicted rating
                u_rec[r] = self.CalcRating(r,u_vec,neighitems)
        return u_rec

In [15]:
# CF 모델에 비해, 차이나는 부분은 행렬뿐으로, 
# 행렬 dimatrix는 아이템 i,j의 차이인 d_ij를 계산하는데 사용된다
# GetKSimItemsperUsers는 K개의 최대 인접 이웃을 dimatrix를 활용하여 찾는다
# 아무도 아이템을 평가 않은경우, default롤 1000을 설정한다

<h3>02 <strong>모델 기반의 협업 필터링</strong>Model-based Collaborative Filtering </h3>
유틸리티 행렬을 사용하여, 사용자의 <strong>아이템 평가 패턴</strong>을 추출한다

<h4>1) <strong>교대 최소 제곱법 (ALS)</strong> Alternative least square </h4>
각 사용자와 아이템을 <strong>차원 K의 특징 공간에 표현</strong>하는, 행렬 R을 분해하는 가장 간단한 방법이다
$$ R \approx PQ^T = \hat R$$
여기서 $P(N \times K)$는 : 특징공간에서 <strong>새로운 사용자 행렬</strong>이고, 

$Q(M \times K)$는 : 특징공간에서 </strong>아이템의 투영(Projection)</strong>이다

이들을 통해, <strong>낮은 차원으로 문제를 축소하여 정규화</strong>한 <strong>비용함수 $J$</strong>를 <strong>최소</strong>로 하는 $P$ 와 
$Q$를 구할 수 있다.
$$ J = min\sum e_{ij}^2 = min\sum Mc_{ij}( r_{ij} - \sum_{k=1}^K P_{ij}q_{kj})^2 + \frac{\lambda}{2}(|p_i|^2 + |q_j^T|^2) $$
$\lambda$는 <strong>정규화 파라미터</strong>로 과적합을 방지한다, 즉 $p_i,q_j^T$에 패널티를 주어 급격한 값의 변화를 방지한다

In [16]:
def ALS(Umatrix, K, iterations=50, l=0.001, tol=0.001):
    nrows, ncols = len(Umatrix), len(Umatrix[0])  
    P, Q = np.random.rand(nrows,K), np.random.rand(ncols,K)
    Qt = Q.T
    err = 0.
    Umatrix = Umatrix.astype(float)
    mask = Umatrix > 0.
    mask[mask==True] = 1
    mask[mask==False] = 0
    mask = mask.astype(np.float64, copy=False)
    
    for it in range(iterations):
        for u, mask_u in enumerate(mask):
            P[u] = np.linalg.solve(np.dot(Qt, np.dot(np.diag(mask_u), Qt.T)) + l*np.eye(K), 
                                np.dot(Qt, np.dot(np.diag(mask_u), Umatrix[u].T))).T
        for i, mask_i in enumerate(mask.T):
            Qt[:,i] = np.linalg.solve(np.dot(P.T, np.dot(np.diag(mask_i), P)) + l*np.eye(K),
                                np.dot(P.T, np.dot(np.diag(mask_i), Umatrix[:,i])))                            
        err = np.sum((mask*(Umatrix - np.dot(P, Qt)))**2)
        if err < tol: break
    return np.round(np.dot(P,Qt),0)

In [17]:
# 코드의 해설
# Mc행렬을 mask로 정의하였다. 변수 l은 정규화 파라미터로 기본값을 (0.001로 설정)
# 최소 제곱의 문제는 linalg.slove 함수를 이용한다
# 확률 내리막 경사법(SGD), 특이값 분해(SVD)보다 정밀도는 떨어지지만, 구현이 쉽고 병렬처리로 속도를 빠르게 하기에 용이하다

<h4>2) <strong>확률 내리막 경사법 (SGD)</strong> Stochastic gradient descent </h4>
$$ R \approx PQ^T = \hat R$$
행렬 $P(N \times K)$와 $Q(M \times K)$는 $K$차원의 잠재특징 곤간에서 사용자와 아이템을 나타내고

근사 평점 $\hat r_{ij}$는 다음과 같이 표현이 가능하다
$\hat r_{ij} = \sum_{k=1}^K$

In [18]:
# Utility-Matrix, K차원의 잠재특징공간
def SGD(Umatrix, K, iterations=100, alpha=0.00001, l=0.001, tol=0.001):
    nrows = len(Umatrix)
    ncols = len(Umatrix[0])  
    P = np.random.rand(nrows,K)
    Q = np.random.rand(ncols,K)
    Qt = Q.T
    cost=-1
    for it in range(iterations):
        for i in range(nrows):
            for j in range(ncols):
                if Umatrix[i][j] > 0:
                    eij = Umatrix[i][j] -np.dot(P[i,:],Qt[:,j])
                    for k in range(K):
                        P[i][k] += alpha*(2*eij*Qt[k][j]-l*P[i][k])
                        Qt[k][j] += alpha*(2*eij*P[i][k]-l*Qt[k][j]) 
        cost = 0
        for i in range(nrows):
            for j in range(ncols):
                if Umatrix[i][j]>0:
                    cost += pow(Umatrix[i][j]-np.dot(P[i,:],Qt[:,j]),2)
                    for k in range(K):
                        cost += float(l/2.0)*(pow(P[i][k],2)+pow(Qt[k][j],2))
        if cost < tol:  break
    return np.round(np.dot(P,Qt),0)

In [ ]:
# SVD 보다 병렬처리는 쉬워서 처리속도는 빠르지만, 정밀도는 떨어진다

<h4>3) <strong>비음수 행렬 분해 (NMK)</strong> Non-negative matrix factorization </h4>
행렬 $R$을 두개의 행렬 $P(N \times K)$와 $Q(M \times K)$ 곱으로 분해하되, 

행렬의 요소가 음수가 아닌 방법들의 그룹으로 이들을 해결하는 방법으로는
1. 투영 경사법 Projection Gradiend
2. 좌표 하강법 Coordinate Descent
3. 비음수 제한 최소 제곱법 Non-negativity constrained least squares

In [21]:
# 좌표 하강법을 활용하여, 비음수 행렬 분해를 한다
from sklearn.decomposition import NMF
def NMF_alg(Umatrix, K, inp = 'none', l = 0.001):
    R_tmp = copy.copy(Umatrix)
    R_tmp = R_tmp.astype(float)
    # Imputation
    if inp != 'none': R_tmp = imputation(inp, Umatrix)
    nmf = NMF(n_components = K , alpha = l)
    P = nmf.fit_transform(R_tmp)
    R_tmp = np.dot(P, nmf.components_)
    return R_tmp

<h4>4) <strong>특이값 분해 (SVD)</strong> Singular value decomposition </h4>
비지도 학습의 SVD는 행렬을 $U, \sum, V$로 분해하여 근사하는 차원 축소 법이고,

추천 시스템 에서는 <strong>행렬분해기법</strong>으로 사용된다

이떄는, 데이터 결측을 처리시 유틸리티행렬의 각 행/열의 평균, 또는 이들의 조합을 사용한다

유틸리티행렬에 SVD를 적용하면서 EM(기대치 최대화)알고리즘을 함께 적용할 수 있다
1. M-step : $\hat R = SVD(\hat R) $ 을 실행
2. E-step : $\hat r_{ij} =  f(n) = \begin{cases} 
    r_{ij} if r_{ij} is-filled-by-the-user \\
    \hat r_{ij} else (missing-data)  \end{cases}$

이 단계는 제곱 오차의 합 $ \sum_{ij} (r_{ij} - \hat r_{ij})^2 $ 이 <strong>허용오차 보다 작아질 떄</strong> 까지 반복한다

In [22]:
from sklearn.decomposition import TruncatedSVD
def SVD(Umatrix,K,inp='none'):
    R_tmp = copy.copy(Umatrix)
    R_tmp = R_tmp.astype(float)
    #imputation
    if inp != 'none':
        R_tmp = imputation(inp,Umatrix)     

    means = np.array([ R_tmp[i][R_tmp[i]>0].mean() for i in range(len(R_tmp))]).reshape(-1,1)
    R_tmp = R_tmp-means
    svd = TruncatedSVD(n_components=K, random_state=4)
    R_k = svd.fit_transform(R_tmp)
    R_tmp = svd.inverse_transform(R_k)
    R_tmp = means+R_tmp
    
    return np.round(R_tmp,0)

In [ ]:
def SVD_EM(Umatrix, K, inp = 'none', iterations = 50, tol = 0.001):
    R_tmp = copy.copy(Umatrix)
    R_tmp = R_tmp.astype(float)
    nrows, ncols = len(Umatrix), len(Umatrix[0])
    # imputation
    if inp != 'none': R_tmp = imputation(inp, Umatrix)
    # define SVD
    svd = TruncatedSVD(n_components = K, random_state = 4)
    err = -1
    for it in range(iterations):
        R_k = svd.fit_transform(R_tmp)  # 1. m-step
        R_tmp = svd.inverse_transform(R_k)
        err = 0 # 2. e-step and error evaluation
        for i in range(nrows):
            for j in range(ncols):
                if Umatrix[i][j]>0:
                    err += pow(Umatrix[i][j]-R_tmp[i][j],2)
                    R_tmp[i][j] = Umatrix[i][j]           
        if err < tol:
            print (it,'toll reached!'); break
    return np.round(R_tmp,0)

In [ ]:
# 함수의 설명
# SVD 는 sklearn에서 제공을 한다
# 이방법은 ALS보단 느리지만, 일반적으로 정확도는 높다 

<h3>03 <strong>콘텐츠 기반의 필터링 (CBF)</strong> Content-Based Filtering </h3>
<strong>아이템의 설명 데이터</strong>에 의존해서 사용자의 특징을 추출한다

아래의 경우 <strong>영화 장르의 특징</strong>을 기반으로 하는 <strong>이진벡터 $m_j$</strong>를 새로 생성한다

(영화 장르가 unknown, action, adventure, animation, children's, comedy, crime, 

documentary, drama, fantasy, film noir, horror, musical, mystery, romance, sci-fi, 

thriller, war, or western 중복여부를 파악하여 <strong>유효값 '1'의 밀도를 높여</strong>, 분석력을 향상시킨다)

<h4> <strong> 1) 이진벡터 $m_j$ </strong></h4>
해당 장르에는 $1$을, 비해당 장르는 $0$을 입력한다

In [27]:
# # 영화목록 추출 (index 라인 번호를 제목 뒤에 덧붙임)
# df_info = pd.read_csv('./data/ml-100k/u.item', 
#                       sep='|', header=None, encoding = "ISO-8859-1")
# movie_list = [df_info[1].tolist()[ index_num ] + ';' + str(index_num+1) 
#               for index_num in range(len(df_info[1].tolist())) ]
# df_out = pd.DataFrame(columns=['user']+movie_list) # 추출한 내용을 DataFrame으로 생성

In [28]:
# Matrix movies's content
movies_list = [int(m.split(';')[-1])       for m in df_out.columns[1:]]
movies_cats = ['unknown','Action','Adventure','Animation','Children\'s','Comedy','Crime','Documentary',
              'Drama','Fantasy','Film-Noir','Horror','Musical','Mystery',
              'Romance','Sci-Fi','Thriller','War','Western']
dfout_movies =  pd.DataFrame(columns=['movie_id'] + movies_cats)
dfout_movies.head()

,movie_id,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western


In [35]:
# 5 는 df_info에 pk, 제목, 출시일, IMBD등 불필요한 자료 건너띄기 위한 값이다.
start_cats_index, cnt = 5, 0  
print('df_info (장르 데이터가 기록된 테이블) : \n', df_info.head(2))
for m in movies_list:
    # .loc[index_num] : 데이터를 1줄씩 list 형태로 입력하여, 테이블을 생성한다
    # df_info 에 정리된 장르 데이터를 위 컬럼에 맞게 재정렬 
    dfout_movies.loc[cnt] = [m] + df_info.iloc[m-1][start_cats_index: ].tolist()
    cnt += 1 
dfout_movies.to_csv('data/movies_content.csv', index = None)
dfout_movies.head(3)

df_info (장르 데이터가 기록된 테이블) : 
    0                 1            2   3   \
0   1  Toy Story (1995)  01-Jan-1995 NaN   
1   2  GoldenEye (1995)  01-Jan-1995 NaN   

                                                  4   5   6   7   8   9  ...  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...   0   0   0   1   1 ...   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...   0   1   1   0   0 ...   

   14  15  16  17  18  19  20  21  22  23  
0   0   0   0   0   0   0   0   0   0   0  
1   0   0   0   0   0   0   0   1   0   0  

[2 rows x 24 columns]


,movie_id,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


<h4> <strong> 2) 아이템 특징 평균 방법 </strong>(Item Features Average Method) </h4>
<strong>사용자 $i$</strong>별로 <strong>영화 장르 선호벡터 $v_i = (v_{i0},...,v_{iG-1})$를 생성</strong>하는 것이 목표다
$$ v_{ig} = \frac{ \sum( r_{ik} - \bar r_i) I_{kg} } {\sum I_{kg}}$$
1. $ v_{ig}$는 사용자 $i$가 본 영화중 장르 $g$에 속하는 영화평점에서, 
   
   평균평점 $\bar r_i$를 뺀 값들의 합을 장르 $g$에 속하는 영화의 갯수로 나눈 값이다
2. $I_{kg}$는 영화$k$가 장르 g에 속하면 1이고, 아니면 0이다

결과 $v_i$는 이진벡터 $m_j$ 와  $\cos$ 유사도로 비교하며, 가장 유사도 높은 영화를 사용자 $i$에게 추천한다

In [ ]:
# class CBF_averageprofile(object):
    def __init__(self, Movies, Movies_list):
        #calc user profiles:
        self.n_features = len(Movies[0])
        self.Movies_list, self.Movies = Movies_list, Movies
        
    def GetRecMovies(self, u_vec, indexs = False):
        #generate user profile
        n_movies = len(u_vec)
        n_features = self.n_features
        mean_u = u_vec[u_vec>0].mean()
        diff_u = u_vec-mean_u
        features_u = np.zeros(n_features).astype(float)
        cnts = np.zeros(n_features)
        for m in range(n_movies):
            if u_vec[m]>0: # u has rated m
                features_u += self.Movies[m]*(diff_u[m])
                cnts += self.Movies[m]
        # Average:
        for m in range(n_features):
            if cnts[m]>0:
                features_u[m] = features_u[m]/float(cnts[m])
        # calc sim:
        sims = np.zeros(n_movies)
        for m in range(n_movies):
            if u_vec[m]==0: # Sim only for movies not yet rated by the user
                sims[m] = sim(features_u,self.Movies[m])
        # order movies
        order_movies_indxs = np.argsort(sims)[::-1] 
        if indexs:
            return order_movies_indxs
        return self.Movies_list[order_movies_indxs]

In [38]:
# Movies_list : 영화 제목을 저장
# Movie : 영화의 특징을 저장
# GetRecMovies 함수 : 사용자 장르 선호벡터 feature_u 생성, 이 벡터와 가장 유사한 Movie (cos 유사도)를 반환한다 

<h4><strong> 3) 정규화된 선형 회귀 분석 방법 </strong> (Regularized linear regression method) </h4>
사용자의 영화 선호도를 <strong>선형 모델의 파라미터</strong>로 학습한다

이는 파라미터 벡터 $\theta _i$의 학습을 위해 정규화된 최소화 문제를 풀어야 한다
$$ min_{\theta _0,...,\theta_{N-1}} \frac{1}{2} \sum_{i=0}^{N-1} 
\sum_{k=0}^{M-1}(\theta_i^T m_j - r_{ij})^2I_{ij} + \frac{\lambda}{2} 
\sum_{i=0}^{N-1} \sum_{k=0}^{G} \theta_{ik}^2$$

<strong>$I_{ij}$</strong>는 사용자 $i$가 영화를 봤으면 1/ 아니면 0이고, $\lambda$는 정규화 파라미터다

파라미터의 해는 <strong>Gradient Descent</strong>를 적용하여 얻을 수 있다

In [40]:
class CBF_regression(object):
    def __init__(self, Movies, Umatrix, alpha = 0.01, l=0.0001, its=50, tol=0.001):
        # calc parameters:
        self.nfeatures = len(Movies[0]) + 1 #intercept
        nusers, nmovies = len(Umatrix), len(Umatrix[0])
        # add intercept col
        movies_feats = np.ones((nmovies, self.nfeatures))
        movies_feats[:,1:] = Movies
        self.movies_feats = movies_feats.astype(float)
        self.Umatrix = Umatrix.astype(float) #set Umatrix as float
        # initialize the matrix:
        Pmatrix = np.random.rand(nusers, self.nfeatures)
        Pmatrix[:,0], err, cost =1., 0., -1
        for it in range(its):
            print('it:', it, ' -- ', cost)
            for u in range(nusers):
                for f in range(self.nfeatures):                    
                    if f==0:#no regularization
                        for m in range(nmovies):
                            if self.Umatrix[u,m]>0:
                                diff = np.dot(Pmatrix[u], self.movies_feats[m]) - self.Umatrix[u, m]
                                Pmatrix[u, f] += -alpha*(diff*self.movies_feats[m][f])
                    else:
                        for m in range(nmovies):
                            if self.Umatrix[u,m]>0:
                                diff = np.dot(Pmatrix[u],self.movies_feats[m])-self.Umatrix[u,m]
                                Pmatrix[u,f] += -alpha*(diff*self.movies_feats[m][f] +l*Pmatrix[u][f])        
                
            cost = 0
            for u in range(nusers):
                for m in range(nmovies):
                    if self.Umatrix[u][m]>0:
                        cost += 0.5*pow(Umatrix[u][m] - np.dot(Pmatrix[u], self.movies_feats[m]), 2)
                for f in range(1, self.nfeatures):
                    cost += float(l/2.0) * (pow(Pmatrix[u][f], 2))
            if cost < tol:
                print('err',cost); break
        self.Pmatrix = Pmatrix
        
    def CalcRatings(self,u_vec):
        s = 0.         #find u_vec
        u_feats = np.zeros(len(self.Pmatrix[0]))
        # in case the user is not present in the utility matrix find the most similar
        for u in range(len(self.Umatrix)):
            # print self.Umatrix[u]
            tmps = sim(self.Umatrix[u], u_vec)
            if tmps > s:
                s = tmps
                u_feats = self.Pmatrix[u]
            if s == 1.: break
        new_vec = np.zeros(len(u_vec))
        for r in range(len(u_vec)):
            if u_vec[r] == 0:
                new_vec[r] = np.dot(u_feats, self.movies_feats[r])
        return new_vec

<br></br>
## 4 추천 시스템 학습을 위한 연관 규칙 association rules
이력 데이터의 추론 방식으로, 광범위한 문제를 해결할 때 사용된다

이 아이디어의 핵심은 <strong>아이템 발생빈도에 대한 통계</strong>를 기반으로, <strong>아이템간의 관계</strong>를 찾는 것이다
<h3>01 <strong>지지도(Support)와 신뢰도(Confidence)</strong></h3>
1. 지지도(Support) : <strong>해당규칙의 발생확률</strong>로, 해당 규칙이 얼마나 의미 있는지를 평가한다.
   
   지지도가 높을수록 해당 조건을 만족하는 다른규칙은 적어진다.
2. 신뢰도(Confidence) : <strong>X가 발생할 떄 Y가 발생할 조건부 확률</strong>을 나타낸다

In [41]:
class AssociationRules(object):
    def __init__(self, Umatrix, Movies_list, min_support=0.1, min_confidence=0.1, likethreshold=3):
        self.min_support = min_support
        self.min_confidence = min_confidence
        self.Movies_list = Movies_list
        # transform utility matrix to sets of liked items
        n_items = len(Umatrix[0])
        trans_actions = []
        for u in Umatrix:
            s = [i     for i in range(len(u))     if  u[i] > likethreshold]
            if len(s) > 0 : trans_actions.append(s)
        # find sets of 2 items
        flat = [item     for sublist in trans_actions     for item in sub_list]
        init_items = map(frozenset,[ [item]      for item in frozenset(flat)])
        set_trans = map(set, trans_actions)
        sets_init, self.dict_sets_support = self.filterSet(set_trans, init_items)
        setlen = 2
        items_tmp = self.combine_lists(sets_init, setlen)
        self.freq_sets, sup_tmp = self.filterSet(set_trans, items_tmp)
        self.dict_sets_support.update(sup_tmp)
        self.ass_matrix = np.zeros((nitems, nitems))
        for freqset in self.freq_sets:
            # print 'freqset',freqset
            list_setitems = [frozenset([item]) for item in freqset]
            # print "freqSet", freqset, 'H1', list_setitems
            self.calc_confidence_matrix(freqset, list_setitems)
        
    def filterSet(self, set_trans, likeditems):
        itemscnt = {}
        for id in set_trans:
            for item in likeditems:
                if item.issubset(id):
                    itemscnt.setdefault(item, 0)
                    itemscnt[item] += 1
        num_items = float(len(set_trans))
        freq_sets = []
        dict_sets = {}
        for key in itemscnt:
            support = itemscnt[key] / num_items
            if support >= self.min_support:
                freq_sets.insert(0, key)
            dict_sets[key] = support
        return freq_sets, dict_sets
        
    def combine_lists(self, freq_sets, setlen):
        setitems_list = []
        nsets = len(freq_sets)
        for i in range(nsets):
            for j in range(i + 1, nsets):
                setlist1 = list(freq_sets[i])[:setlen - 2]
                setlist2 = list(freq_sets[j])[:setlen - 2]
                if set(setlist1) == set(setlist2):
                    setitems_list.append(freq_sets[i].union(freq_sets[j]))
        return setitems_list
        
    def calc_confidence_matrix(self, freqset, list_setitems):
        for target in list_setitems:
            confidence = self.dict_sets_support[freqset] / self.dict_sets_support[freqset - target]
            if confidence >= self.min_confidence:
                self.ass_matrix[list(freqset - target)[0]][list(target)[0]] = confidence
                
    def GetRecItems(self, u_vec, indxs = False):
        vec_recs = np.dot(u_vec, self.ass_matrix)
        sorted_weight = np.argsort(vec_recs)
        seen_indexs = [index    for index in  range(len(u_vec))   if u_vec[indx]>0]
        seen_movies = np.array(self.Movies_list)[seen_indexs]
        #remove seen items
        recitems = np.array(self.Movies_list)[sorted_weight]
        recitems = [m for m in recitems if m not in seen_movies]
        if indexs:
            vec_recs[seen_indexs]=-1
            recs_vec = np.argsort(vec_recs)[::-1][np.argsort(vec_recs)>0]
            return recs_vec
        return recitems[::-1]

<h3>02 <strong>$Log$ 우도비 추천 시스템</strong></h3>
Log-likelihood ratios recommendation system method

두개의 이벤트가 얼마나 함께 발생하는가에 대한 척도이다.

이는 두 이벤트가 <strong>정규분포 예측값</strong>보다 더 높은 빈도로 동시 발생하는 곳을 찾는다

In [46]:
class LogLikelihood(object):
    def __init__(self, Umatrix, Movieslist, likethreshold=3):
        self.Movieslist = Movieslist
        #calculate loglikelihood ratio for each pair
        self.nusers = len(Umatrix)
        self.Umatrix = Umatrix
        self.likethreshold = likethreshold
        self.likerange = range(self.likethreshold + 1 , 5 + 1)
        self.dislikerange = range(1,self.likethreshold + 1)
        self.loglikelihood_ratio()

    def calc_k(self, a, b):
        tmpk = [[0    for j in range(2)]    for i in range(2)]
        for ratings in self.Umatrix:
            if ratings[a] in self.likerange and ratings[b] in self.likerange:
                tmpk[0][0] += 1
            if ratings[a] in self.likerange and ratings[b] in self.dislikerange:
                tmpk[0][1] += 1
            if ratings[a] in self.dislikerange and ratings[b] in self.likerange:
                tmpk[1][0] += 1
            if ratings[a] in self.dislikerange and ratings[b] in self.dislikerange:
                tmpk[1][1] += 1
        return tmpk
        
    def calc_llr(self,k_matrix):
        Hcols = Hrows = Htot = 0.0
        if  sum(k_matrix[0])+sum(k_matrix[1]) == 0:
            return 0.
        invN = 1.0 / (sum(k_matrix[0]) + sum(k_matrix[1])) 
        for i in range(0,2):
            if((k_matrix[0][i]+k_matrix[1][i])!=0.0):
                Hcols += invN * (k_matrix[0][i] + k_matrix[1][i]) *\
                         math.log((k_matrix[0][i]+k_matrix[1][i]) * invN ) #sum of rows
            if((k_matrix[i][0]+k_matrix[i][1])!=0.0):
                Hrows += invN * (k_matrix[i][0] + k_matrix[i][1]) *\
                         math.log((k_matrix[i][0] + k_matrix[i][1]) * invN )#sum of cols
            for j in range(0,2):
                if(k_matrix[i][j] != 0.0):
                    Htot += invN * k_matrix[i][j] * math.log(invN * k_matrix[i][j])
        return 2.0 * (Htot-Hcols-Hrows) / invN

    def loglikelihood_ratio(self):
        nitems = len(self.Movieslist)
        self.items_llr= pd.DataFrame(np.zeros((nitems, nitems))).astype(float)
        for i in range(nitems):
            for j in range(nitems):
                if(j >= i):
                    tmpk = self.calc_k(i, j)
                    self.items_llr.ix[i, j] = self.calc_llr(tmpk)
                else:
                    self.items_llr.ix[i, j] = self.items_llr.iat[j, i]
        
    def GetRecItems(self, u_vec, indexs = False):
        items_weight = np.dot(u_vec, self.items_llr)
        sorted_weight = np.argsort(items_weight)
        seen_indexs = [index   for index in range(len(u_vec)) if u_vec[index]>0]
        seen_movies = np.array(self.Movies_list)[seen_indexs]
        #remove seen items
        rec_items = np.array(self.Movies_list)[sorted_weight]
        rec_items = [m     for m in rec_items    if m not in seen_movies]
        if indexs:
            items_weight[seen_indexs] = -1
            recs_vec = np.argsort(items_weight)[::-1][np.argsort(items_weight)>0]
            return recs_vec
        return rec_items[::-1]

<h3>03 <strong>하이브리드 추천 시스템</strong></h3>
결과를 좋게 만들기 위하여 <strong>CBF</strong>와 <strong>CF</strong>를 합쳐서 하나의 추천기를 만든다

ex) 아이템 특징 CBF와, 사용자 기반 CF를 합쳐서 두개의 하이브리드 특징조합을 구현

1. 가중(Weighted) : CB와 CBF의 예측 평점을 <strong>가중 평균</strong>으로 합친다 
2. 혼합(Mixed) :  CB와 CBF의 예측 영화들을, <strong>하나의 목록</strong>으로 합병한다
3. 교대(Switched) : <strong>특정조건</strong>으로 CF, 혹은 CBF 예측을 사용한다
4. 특징조합(Feature Combination) : <strong>CB와 CBF의 특징을 함께 고려</strong>해, 가장 유사한 아이템을 찾는다
5. 특징증가(Feature augmentation) : 특징조합과 유사하나, <strong>추가 특징을 사용</strong>하여 평점을 예측하고, 
   
   이 평점을 이용해 주 추천기의 목록을 생성한다
   
   ex) <strong>Content-Boosted Collaborative Filtering</strong> 학습 : 아이템 특징 컨텐츠 기반의 모델을 이용해, 
   
   평가되지 않은 영화의 평점을 학습한 후, 추천을 정의할 떄에는 협업방식을 사용한다
 

In [48]:
class Hybrid_cbf_cf(object):
    def __init__(self,Movies,Movieslist,Umatrix):
        #calc user profiles:
        self.nfeatures = len(Movies[0])
        self.Movieslist = Movieslist 
        self.Movies = Movies.astype(float)
        self.Umatrix_mfeats = np.zeros((len(Umatrix),len(Umatrix[0])+self.nfeatures))
        means = np.array([ Umatrix[i][Umatrix[i]>0].mean() for i in xrange(len(Umatrix))]).reshape(-1,1)
        diffs = np.array([ [Umatrix[i][j]-means[i] if Umatrix[i][j]>0 else 0. 
                            for j in xrange(len(Umatrix[i]))  ] for i in xrange(len(Umatrix))])
        self.Umatrix_mfeats[:,:len(Umatrix[0])] = Umatrix#diffs
        self.nmovies = len(Movies)
        #calc item features for each user
        for u in xrange(len(Umatrix)):
            u_vec = Umatrix[u]
            self.Umatrix_mfeats[u,len(Umatrix[0]):] = self.GetUserItemFeatures(u_vec)
            
    def GetUserItemFeatures(self,u_vec):
        mean_u = u_vec[u_vec > 0].mean()
        # diff_u = u_vec-mean_u
        features_u = np.zeros(self.nfeatures).astype(float)
        cnts = np.zeros(self.nfeatures)
        for m in range(self.nmovies):
            if u_vec[m] > 0 : #u has rated m
                features_u += self.Movies[m] * u_vec[m] #self.Movies[m]*(diff_u[m])
                cnts += self.Movies[m]
        #average:
        for m in range(self.nfeatures):
            if cnts[m]>0: features_u[m] = features_u[m]/float(cnts[m])
        return features_u
    def CalcRatings(self,u_vec,K):
        def FindKNeighbours(r,data,K):
            neighs, cnt = [], 0
            for u in range(len(data)):
                if data[u,r] > 0  and  cnt < K :
                    neighs.append(data[u])   
                    cnt += 1 
                elif cnt == K: break
            return np.array(neighs)
        
        def CalcRating(u_vec,r,neighs):
            rating, den = 0., 0.
            for j in range(len(neighs)):
                rating += neighs[j][-1] *\
                          float(neighs[j][r] - neighs[j][neighs[j]>0][:-1].mean())
                den += abs( neighs[j][-1] )
            if den > 0: rating = np.round(u_vec[u_vec > 0].mean()+(rating / den), 0)
            else:       rating = np.round(u_vec[u_vec > 0].mean(), 0)
            if rating > 5:   return 5.
            elif rating < 1: return 1.
            return rating
        # add similarity col
        nrows = len(self.Umatrix_mfeats)
        ncols = len(self.Umatrix_mfeats[0])
        data_sim = np.zeros((nrows,ncols+1))
        data_sim[ : , : -1] = self.Umatrix_mfeats
        u_rec = np.zeros(len(u_vec))
        # calc similarities:
        mean = u_vec[u_vec > 0].mean()
        u_vec_feats = u_vec 
        # np.array([u_vec[i]-mean if u_vec[i]>0 else 0 for i in range(len(u_vec))])
        u_vec_feats = np.append(u_vec_feats, self.GetUserItemFeatures(u_vec))
        
        for u in range(nrows):
            if np.array_equal(data_sim[u,:-1],u_vec) == False: #list(data_sim[u,:-1]) != list(u_vec):
                data_sim[u,ncols] = sim(data_sim[u,:-1],u_vec_feats)
            else: data_sim[u,ncols] = 0.
        # order by similarity:
        data_sim = data_sim[ data_sim[ :, ncols].argsort()][::-1]

        #find the K users for each item not rated:
        for r in range(self.nmovies):
            if u_vec[r] == 0:
                neighs = FindKNeighbours(r, data_sim, K)
                #calc the predicted rating
                u_rec[r] = CalcRating(u_vec, r, neighs)
        return u_rec

In [ ]:
class Hybrid_svd(object):
    def __init__(self,Movies,Movieslist,Umatrix,K,inp):
        #calc user profiles:
        self.nfeatures = len(Movies[0])
        self.Movieslist = Movieslist 
        self.Movies = Movies.astype(float)
        
        R_tmp = copy.copy(Umatrix)
        R_tmp = R_tmp.astype(float)
        #imputation
        
        if inp != 'none':
            R_tmp = imputation(inp,Umatrix)
        Umatrix_mfeats = np.zeros((len(Umatrix),len(Umatrix[0])+self.nfeatures))
        means = np.array([ Umatrix[i][Umatrix[i]>0].mean() for i in xrange(len(Umatrix))]).reshape(-1,1)
        diffs = np.array([ [float(Umatrix[i][j]-means[i]) 
                            if Umatrix[i][j]>0 else float(R_tmp[i][j]-means[i]) for j in xrange(len(Umatrix[i]))  ] 
                          for i in xrange(len(Umatrix))])
        Umatrix_mfeats[:,:len(Umatrix[0])] = diffs#R_tmp
        self.nmovies = len(Movies)
        #calc item features for each user
        for u in xrange(len(Umatrix)):
            u_vec = Umatrix[u]
            Umatrix_mfeats[u,len(Umatrix[0]):] = self.GetUserItemFeatures(u_vec)
        
        #calc svd
        svd = TruncatedSVD(n_components=K, random_state=4)
        R_k = svd.fit_transform(Umatrix_mfeats)
        R_tmp = means+svd.inverse_transform(R_k)
        self.matrix = np.round(R_tmp[:,:self.nmovies],0)
        
        
    def GetUserItemFeatures(self,u_vec):
        mean_u = u_vec[u_vec>0].mean()
        diff_u = u_vec-mean_u
        features_u = np.zeros(self.nfeatures).astype(float)
        cnts = np.zeros(self.nfeatures)
        for m in xrange(self.nmovies):
            if u_vec[m]>0:#u has rated m
               features_u += self.Movies[m]*(diff_u[m])#self.Movies[m]*u_vec[m]
               cnts += self.Movies[m]
        #average:
        for m in xrange(self.nfeatures):
            if cnts[m]>0:
               features_u[m] = features_u[m]/float(cnts[m])
        return features_u